Import packages

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F

Download and normalize the data

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010]
    )
])
dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

100%|██████████| 170M/170M [00:19<00:00, 8.68MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
image, label = dataset[0]
print(f"Image shape: {image.shape}")
print(f"Label: {label}")

Image shape: torch.Size([3, 32, 32])
Label: 6


Split the data into train, test and validation set

In [ ]:
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = int(0.15 * len(dataset))

# Split the dataset
Xtr, Xval, Xts = random_split(dataset, [train_size, val_size, test_size])

Examine the dataset

In [ ]:
print(train_size)
print(val_size)
print(test_size)
print(f"Number of training data: {len(Xtr)}")
print(f"Number of testing data: {len(Xts)}")
print(f"Number of validation data: {len(Xval)}")

35000
7500
7500
Number of training data: 35000
Number of testing data: 7500
Number of validation data: 7500


Create data loaders for each set

In [ ]:
Xtr_loader = DataLoader(Xtr, batch_size=64, shuffle=True)
Xts_loader = DataLoader(Xts, batch_size=64, shuffle=False)
Xval_loader = DataLoader(Xval, batch_size=64, shuffle=False)

Initilize the model

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

Build the neural network

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)
        # self.linear = nn.Linear(256, 10)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        # Add dropout before linear layer
        self.dropout = nn.Dropout(0.3)
        out = self.linear(out)
        return out

In [ ]:
def ResNet18():
    return ResNet(BasicBlock, [1, 1, 1, 1])

## Train the model

First, configure the device

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Set the hyperparameters

In [ ]:
num_classes = 10
num_epochs = 20
batch_size = 16
learning_rate = 0.01

Setup model

In [ ]:
model = ResNet18().to(device)
#Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)

#Train the model
total_step = len(Xtr_loader)
print(total_step)

# See the total number of trainable parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total Trainable Parameters: {total_params}")

547


Now, training

In [ ]:
import gc

for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(Xtr_loader):
    #Move tensors to the configured device
    images = images.to(device)
    labels = labels.to(device)

    #Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    #Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    del images, labels, outputs
    torch.cuda.empty_cache()
    gc.collect()

    print ('Epoch [{}/{}], Loss: {:.4f}'
                    .format(epoch+1, num_epochs, loss.item()))

    #Validation
    with torch.no_grad():
      correct = 0
      total = 0
      for images, labels in Xval_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))

## Testing

In [ ]:
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in Xts_loader:
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      del images, labels, outputs

      print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))